# Introduction
This is a brief introduction to the jupyterlab widget wrapper for displaying diagrams using the Eclipse Layout Kernel - [elkjs](https://github.com/kieler/elkjs). 

In [1]:
import elk

# ElkDiagram
The `ElkDiagram` is a lower level widget that accepts and validates a dictionary against the Elk JSON Schema.

In [2]:
w = elk.ElkDiagram()

In [3]:
w.value = {
  'id': "root",
  'properties': { 'algorithm': 'layered' },
  'children': [
    { 'id': "n1", 'width': 30, 'height': 30 },
    { 'id': "n2", 'width': 30, 'height': 30 },
    { 'id': "n3", 'width': 30, 'height': 30 }
  ],
  'edges': [
    { 'id': "e1", 'sources': [ "n1" ], 'targets': [ "n2" ] },
    { 'id': "e2", 'sources': [ "n1" ], 'targets': [ "n3" ] } 
  ]
}

In [4]:
w

ElkDiagram(value={'id': 'root', 'properties': {'algorithm': 'layered'}, 'children': [{'id': 'n1', 'width': 30,…

## Centering Diagram
Center of root element

In [5]:
w.center("root")

Center on multiple elements

In [6]:
w.center(["n3", "n2"])

## Selection
Select a single element

In [7]:
w.selected = ['n1']

Select multiple elements

In [8]:
w.selected = ["n2", "n3"]

Clear selection

In [9]:
w.selected = []

## Hovering 
The element currently being hovered is also traited

In [10]:
w.hovered = "n2"

In [11]:
w.hovered

'n2'

## Linking Diagrams
Example of using one diagram's hovered state to control another diagram's view

In [12]:
import ipywidgets as W

w2 = elk.ElkDiagram(value=w.value)

def centering(*args):
    if w.hovered != "root":
        w2.center(w.hovered)
w.observe(centering, "hovered")

W.HBox([w, w2])


# Elk Transformer
A transformer object that will convert some input source into valid Elk Json.
This example using a transformer around a networkx graph.

In [13]:
import networkx as nx

from elk.nx import XELK
from elk.tools import ToggleCollapsedBtn, ToolButton

## Flat structure
TODO: There should be an option to specify if ports should be created or only connect edges between the nodes

In [14]:
import networkx as nx
import ipywidgets as W

from elk import ElkDiagram
from elk.nx import XELK


G = nx.MultiDiGraph()

# connections
G.add_nodes_from(['n1', 'n2', 'n3'])
G.add_edge('n1', 'n2')
G.add_edge('n1', 'n2')
G.add_edge('n2', 'n3')
G.add_edge('n1', 'n3')


xelk = XELK(source=(G, None))
ew = ElkDiagram(value=xelk.to_dict())
ew

ElkDiagram(value={'id': 'root', 'children': [{'id': 'n1', 'height': 18.0, 'labels': [{'id': 'n1_label_n1', 'te…

## Hierarchical Diagram with Ports 

In [15]:
tree = nx.DiGraph()  # hierarchical graph
G = nx.MultiDiGraph()  # port connections

# heirarchy
tree.add_nodes_from(['n0', 'n1', 'n2', 'n3'])
tree.add_edge('n1', 'n2')
tree.add_edge('n0', 'n1')

# connections
G.add_nodes_from(['n0', 'n1', 'n2', 'n3'])
G.add_edge('n1', 'n2', sourcePort='x', targetPort='x')
G.add_edge('n1', 'n2', port='y')
G.add_edge('n2', 'n3', sourcePort='z', targetPort='x')
G.add_edge('n2', 'n3', port='x', targetPort='w')
G.add_edge('n1', 'n3', port='y')
G.add_edge('n1', 'n3', port='z')

xelk = XELK(source=(G, tree))
ew = elk.ElkDiagram(value=xelk.to_dict())
ew

ElkDiagram(value={'id': 'root', 'children': [{'id': 'n0', 'children': [{'id': 'n1', 'children': [{'id': 'n2', …

In [16]:

toggle = W.Button(description="Toggle Collapsed")
out = W.Output()

def toggle_node(widget):
    with out:
        for element_id in ew.selected:
            print(element_id)
            if element_id in tree:
                for child in tree.neighbors(element_id):
                    state = tree.nodes[child].get('hidden', False)
                    tree.nodes[child]['hidden'] = not state
                ew.value = xelk.to_dict()
    
toggle.on_click(toggle_node)
W.VBox([ew, toggle])

# Hierarchical Example Using the Elk App
This is a higher level widget for interacting with Elk Transformers and Elk Diagrams

In [17]:
from elk import Elk
import ipywidgets as W
import traitlets as T
out = W.Output()
class ToggleCollapsedBtn(ToolButton):

    @T.default("description")
    def _default_description(self):
        return "Toggle Collapsed"

    def handler(self, *args):
        with out:
            diagram:ElkDiagram = self.app.diagram
            transformer:XELK = self.app.transformer
            graph, tree = transformer.source
            # tree = self.app.source
            for element_id in diagram.selected:
                print(element_id)
                if element_id in tree:
                    for child in tree.neighbors(element_id):
                        state = tree.nodes[child].get('hidden', False)
                        tree.nodes[child]['hidden'] = not state
                        print(child, not state)
                    diagram.value = transformer.to_dict()


ek = Elk(transformer=xelk)
toggle = ToggleCollapsedBtn(app=ek)
W.VBox([ek, toggle, out])

## Custom Styling For Collaping Hierarchical Example
The style dictionary takes the first key and adds a parent selector based on the current `StyleWidget`'s id. This helps to namespace the css selectors but also means that the leading space in the style dictionary keys is intentional and is a descendant selector from the root.

The `.slack-port` and `.slack-edge` are custom css classes applied by the XELK transformer when collapsing a node causes either a source or destination of a tunneling edges to disapear requiring the introduction of a new edge and port at a higher level.

In [18]:
ek.style = {
    " rect": {'opacity':'.75'},
    " .slack-port>rect": {'fill':'red', 'opacity':'.25'},
    " .slack-edge>path": {'stroke':'red', 'opacity':'.25'},
    " .slack-edge>path.edge.arrow":{'fill':'red', 'opacity':'.25'}
}